In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sos4hjb.polynomials import Variable, MonomialVector, ChebyshevVector, Polynomial
from sos4hjb.optimization.cvx import SosProgram
from sos4hjb.plot_utils import level_plot

# System dynamics

In [ ]:
# State with limits.
x = Variable.multivariate('x', 2)
xlim = np.array([1, 1])
xobj = xlim / 3
x_m = [MonomialVector.make_polynomial(xi) for xi in x]
xlim_m = [MonomialVector.make_polynomial(xlimi) for xlimi in xlim]
X = [(xi + xlimi) * (xlimi - xi) for xi, xlimi in zip(x_m, xlim_m)]

# Input with limits.
u = Variable('u')
ulim = 1
u_m = MonomialVector.make_polynomial(u)
ulim_m = MonomialVector.make_polynomial(ulim)
U = (u_m + ulim_m) * (ulim_m - u_m)

# Dynamics.
f = [
    2 * x_m[0] ** 3 + x_m[0] ** 2 * x_m[1] - 6 * x_m[0] * x_m[1] ** 2 + 5 * x_m[1] ** 3,
    u_m
]

# Running cost.
l = x_m[0] ** 2 + x_m[1] ** 2 + 5 * u_m ** 2

# Lower bound on the value function (monomial basis)

In [ ]:
def value_function_lower_bound(f, l, X, U, degree):
    
    # Set up SOS program.
    prog = SosProgram()
    vector_type = type(f[0].vectors()[0])
    basis = vector_type.construct_basis(x, degree, odd=False)
    J = prog.add_polynomial(basis)[0]

    # Maximize volume beneath the value function.
    Jint = J.definite_integral(x, - xobj, xobj)
    prog.add_linear_cost(- Jint.to_scalar())

    # S-procedure for the state limits.
    basis = vector_type.construct_basis(x + [u], degree // 2)
    Sprocedure = Polynomial({})
    for Xi in X:
        lamxi = prog.add_even_sos_polynomial(basis)[0]
        Sprocedure += lamxi * Xi

    # S-procedure for the input limits.
    lamu = prog.add_even_sos_polynomial(basis)[0]
    Sprocedure += lamu * U

    # Bellman inequality.
    Jdot = sum(J.derivative(xi) * f[i] for i, xi in enumerate(x))
    prog.add_sos_constraint(Jdot + l - Sprocedure)

    # Value function nonpositive in the origin.
    prog.add_linear_constraint(J({xi: 0 for xi in x}) <= 0)

    # Solve and retrieve result.
    prog.solve()
    Jlb = prog.substitute_minimizer(J)
    obj = - prog.minimum()
    
    return Jlb, obj

In [ ]:
# Solve for increasing degree.
degrees = np.arange(1, 6) * 2
Jlb = {d: value_function_lower_bound(f, l, X, U, d) for d in degrees}

In [ ]:
# Plot solution.
def plot_value_function(Jlb):
    for d in degrees:
        plt.figure()
        label = r'$J_{\mathrm{lb}}$'
        title = f'Degree {d} (objective {round(Jlb[d][1], 3)})'
        level_plot(Jlb[d][0], - xobj, xobj, zlabel=label, title=title)
plot_value_function(Jlb)

# Lower bound on the value function (Chebyshev basis)

In [ ]:
# Translate polynomial data in Chebyshev basis.
f_c = [fi.in_chebyshev_basis() for fi in f]
l_c = l.in_chebyshev_basis()
X_c = [Xi.in_chebyshev_basis() for Xi in X]
U_c = U.in_chebyshev_basis()

# Solve for increasing degree.
Jlb_c = {d: value_function_lower_bound(f_c, l_c, X_c, U_c, d) for d in degrees}

In [ ]:
# Plot solution with Chebyshev basis.
plot_value_function(Jlb_c)